# trmi3_dc_pca

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import reduce
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('axerunt_test.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
from dask import delayed

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, is_valid, is_type, null_fn, identity_fn, pd_abs, pd_is_empty, ser_range_center_clip
from common_util import load_json, dump_json, pd_slot_shift, arr_nonzero, df_rows_in_year, get_variants, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import GRAPHS_DIR, HISTORY_DIR, TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import get_rm_keychain, process_transform
from runt_util import RUNT_TYPE_MAPPING, RUNTFormatError, get_ser_fn

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.0
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'impo

CRITICAL:root:script location: /home/kev/crunch/mutate/axerunt_test.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Init DataAPI

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
graphs, transforms = get_graphs(), get_transforms()

## #Set Transform

In [8]:
data_rcs, data_dfs = DataAPI.axe_load(['dc', 'dc'])

In [21]:
ret_rcs, ret_dfs = DataAPI.axe_load(['dret', 'dret_pba_hoc'])

In [27]:
list(data_dfs.keys())

[['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_news_forecast_dc',
  'trmi3_etf3_news_forecast_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_news_fundamental_dc',
  'trmi3_etf3_news_fundamental_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_news_sentiment_dc',
  'trmi3_etf3_news_sentiment_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_social_forecast_dc',
  'trmi3_etf3_social_forecast_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_social_fundamental_dc',
  'trmi3_etf3_social_fundamental_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_etf3_social_sentiment_dc',
  'trmi3_etf3_social_sentiment_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_mkt3_news_bank_dc',
  'trmi3_mkt3_news_bank_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_mkt3_news_bond_dc',
  'trmi3_mkt3_news_bond_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_mkt3_news_stock_dc',
  'trmi3_mkt3_news_stock_dc'],
 ['dow_jones',
  'dc',
  'dc',
  'trmi3_mkt3_social_bank_dc',
  'trmi3_mkt3_social_bank_dc'],
 ['dow_jones',
  '

In [29]:
list(ret_dfs.keys())

[['dow_jones',
  'dret',
  'dret_pba_hoc',
  'pba_hoc_hdxret_dret',
  'pba_hoc_hdxret_dret'],
 ['nasdaq_100',
  'dret',
  'dret_pba_hoc',
  'pba_hoc_hdxret_dret',
  'pba_hoc_hdxret_dret'],
 ['russell_2000',
  'dret',
  'dret_pba_hoc',
  'pba_hoc_hdxret_dret',
  'pba_hoc_hdxret_dret'],
 ['sp_500',
  'dret',
  'dret_pba_hoc',
  'pba_hoc_hdxret_dret',
  'pba_hoc_hdxret_dret']]

In [ ]:
PCA(n_components=5, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power=’auto’, random_state=None)

In [26]:
list(ret_dfs.keys())[-1]

['sp_500',
 'dret',
 'dret_pba_hoc',
 'pba_hoc_hdxret_dret',
 'pba_hoc_hdxret_dret']